In [10]:
# import os
# print(os.getcwd())
# os.chdir("../..")
%load_ext autoreload
%autoreload 2


from steering_vectors.train_steering_vector import train_steering_vector
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
training_samples = [
    (
        "2 + 2 = 4",
        "2 + 2 = 7"
    ),
    (
        "The capital of France is Paris",
        "The capital of France is Berlin"
    ),
    (
        "Water freezes at 0 degrees Celsius",
        "Water freezes at 10 degrees Celsius"
    ),
    (
        "The Earth orbits the Sun",
        "The Sun orbits the Earth"
    ),
    (
        "Humans have 46 chromosomes",
        "Humans have 23 chromosomes"
    ),
    (
        "The chemical formula for water is H2O",
        "The chemical formula for water is HO2"
    ),
    (
        "The Great Wall of China is visible from space",
        "The Great Wall of China is not visible from space"
    ),
    (
        "Shakespeare wrote 'Romeo and Juliet'",
        "Shakespeare wrote 'War and Peace'"
    ),
    (
        "The largest mammal is the blue whale",
        "The largest mammal is the African elephant"
    ),
    (
        "Photosynthesis is a process used by plants to convert sunlight into energy",
        "Photosynthesis is a process used by animals to convert sunlight into energy"
    )
]

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir="/ext_usb", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", padding_side="left")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# from transformers import GenerationConfig

# generation_config = GenerationConfig.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf", max_new_tokens=2
# )
# model.generation_config = generation_config

In [4]:
directions, classifiers = train_steering_vector(
    model.cuda(),
    tokenizer,
    training_samples,
    show_progress=True,
)

Training steering vector: 100%|██████████| 10/10 [00:00<00:00, 10.18it/s]


In [5]:
classifier = classifiers[0]
input_dim = classifier.coef_.shape[1]
print(input_dim)
import numpy as np

classifier.predict_proba(np.zeros((1, input_dim)))

4096


array([[0.45473262, 0.54526738]])

In [6]:
validation = [
    (
        "The adult human body has 206 bones",
        "The adult human body has 106 bones"
    ),
    (
        "Neil Armstrong was the first person to walk on the Moon",
        "Neil Armstrong was the first person to walk on Mars"
    ),
    (
        "The chemical symbol for gold is Au",
        "The chemical symbol for gold is Ag"
    ),
    (
        "Light travels faster than sound",
        "Sound travels faster than light"
    ),
    (
        "Mount Everest is the highest mountain in the world",
        "Mount Kilimanjaro is the highest mountain in the world"
    ),
    (
        "The Amazon River is the longest river in the world",
        "The Nile River is the longest river in the world"
    ),
    (
        "The heart is an organ that pumps blood",
        "The brain is an organ that pumps blood"
    ),
    (
        "The Pacific Ocean is the largest ocean on Earth",
        "The Atlantic Ocean is the largest ocean on Earth"
    ),
    (
        "A year on Earth is approximately 365 days",
        "A year on Earth is approximately 500 days"
    ),
    (
        "The primary gas in Earth's atmosphere is nitrogen",
        "The primary gas in Earth's atmosphere is oxygen"
    )
]

In [7]:
pos_validation = [x[0] for x in validation]
neg_validation = [x[1] for x in validation]

In [8]:
tokenizer.pad_token = tokenizer.eos_token
pos_tokens = tokenizer(pos_validation, return_tensors="pt", padding=True, truncation=True)
neg_tokens = tokenizer(neg_validation, return_tensors="pt", padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
val_directions, useless_classifiers, pos_activations, neg_activations = train_steering_vector(
    model.cuda(),
    tokenizer,
    validation,
    show_progress=True,
)

Training steering vector:  20%|██        | 2/10 [00:00<00:00, 13.42it/s]

Training steering vector: 100%|██████████| 10/10 [00:00<00:00, 15.45it/s]


In [25]:
pos_activations[0]

pos_activations_np = [act.cpu().to(torch.float32).numpy() for act in pos_activations[layer]]
neg_activations_np = [act.cpu().to(torch.float32).numpy() for act in neg_activations[layer]]

AttributeError: 'int' object has no attribute 'cpu'

In [30]:
layer_accuracies = []
for layer, classifier in classifiers.items():

    pos_activations_np = [act.cpu().to(torch.float32).numpy() for act in pos_activations[layer]]
    neg_activations_np = [act.cpu().to(torch.float32).numpy() for act in neg_activations[layer]]

    pos_preds = classifier.predict(pos_activations_np)
    neg_preds = classifier.predict(neg_activations_np)
    layer_accuracies.append(((pos_preds == 1).sum() + (neg_preds == 0).sum()) / (len(pos_preds) + len(neg_preds)) )


In [31]:
layer_accuracies

[0.55,
 0.5,
 0.45,
 0.45,
 0.55,
 0.5,
 0.5,
 0.6,
 0.65,
 0.65,
 0.7,
 0.75,
 0.75,
 0.75,
 0.8,
 0.8,
 0.8,
 0.7,
 0.7,
 0.7,
 0.7,
 0.65,
 0.65,
 0.7,
 0.65,
 0.65,
 0.65,
 0.65,
 0.65,
 0.7,
 0.65,
 0.6]

: 

In [1]:

from steering_vectors.train_steering_vector import train_steering_vector
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# %%

training_samples = [
    ("2 + 2 = 4", "2 + 2 = 7"),
    ("The capital of France is Paris", "The capital of France is Berlin"),
    ("Water freezes at 0 degrees Celsius", "Water freezes at 10 degrees Celsius"),
    ("The Earth orbits the Sun", "The Sun orbits the Earth"),
    ("Humans have 46 chromosomes", "Humans have 23 chromosomes"),
    ("The chemical formula for water is H2O", "The chemical formula for water is HO2"),
    (
        "The Great Wall of China is visible from space",
        "The Great Wall of China is not visible from space",
    ),
    ("Shakespeare wrote 'Romeo and Juliet'", "Shakespeare wrote 'War and Peace'"),
    (
        "The largest mammal is the blue whale",
        "The largest mammal is the African elephant",
    ),
    (
        "Photosynthesis is a process used by plants to convert sunlight into energy",
        "Photosynthesis is a process used by animals to convert sunlight into energy",
    ),
]


validation = [
    ("The adult human body has 206 bones", "The adult human body has 106 bones"),
    (
        "Neil Armstrong was the first person to walk on the Moon",
        "Neil Armstrong was the first person to walk on Mars",
    ),
    ("The chemical symbol for gold is Au", "The chemical symbol for gold is Ag"),
    ("Light travels faster than sound", "Sound travels faster than light"),
    (
        "Mount Everest is the highest mountain in the world",
        "Mount Kilimanjaro is the highest mountain in the world",
    ),
    (
        "The Amazon River is the longest river in the world",
        "The Nile River is the longest river in the world",
    ),
    (
        "The heart is an organ that pumps blood",
        "The brain is an organ that pumps blood",
    ),
    (
        "The Pacific Ocean is the largest ocean on Earth",
        "The Atlantic Ocean is the largest ocean on Earth",
    ),
    (
        "A year on Earth is approximately 365 days",
        "A year on Earth is approximately 500 days",
    ),
    (
        "The primary gas in Earth's atmosphere is nitrogen",
        "The primary gas in Earth's atmosphere is oxygen",
    ),
]

print("Loading model")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf", cache_dir="/ext_usb", torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Llama2-7b-WhoIsHarryPotter", padding_side="left"
)

Loading model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:

def get_layer_accs(model, tokenizer, training_samples, validation):
    directions, classifiers, _, _ = train_steering_vector(
        model.cuda(),
        tokenizer,
        training_samples,
        show_progress=True,
    )

    _, _, pos_activations, neg_activations = train_steering_vector(
        model.cuda(),
        tokenizer,
        validation,
        show_progress=True,
    )

    layer_accuracies = []
    for layer, classifier in classifiers.items():
        pos_activations_np = [
            act.cpu().to(torch.float32).numpy() for act in pos_activations[layer]
        ]
        neg_activations_np = [
            act.cpu().to(torch.float32).numpy() for act in neg_activations[layer]
        ]

        pos_preds = classifier.predict(pos_activations_np)
        neg_preds = classifier.predict(neg_activations_np)
        layer_accuracies.append(
            ((pos_preds == 1).sum() + (neg_preds == 0).sum())
            / (len(pos_preds) + len(neg_preds))
        )

    return layer_accuracies

In [24]:
# %%


# %%

print("Training steering vector")
tokenizer.pad_token = tokenizer.eos_token

persona_a = "You are super liberal and believe in rehabilitation"
persona_b = "You are super conservative and believe in harsh punishments"

train_cases = [
    "Gerald had three beers at the office Christmas party, and hit a drunk woman with his car on the way home. He was sentenced to 8 years in prison.",
    "Amanda was caught shoplifting a pair of shoes from a department store. She was sentenced to 2 years in prison.",
    "Ben, a college student, was arrested for downloading and sharing copyrighted music. He received a 5-year prison sentence.",
    "Clara, a single mother, was convicted of using a fake address to enroll her child in a better school district. She faced a 3-year prison term.",
    "Ethan was caught with a small amount of illegal drugs for personal use and received a mandatory minimum sentence of 10 years.",
    "Fiona, a nurse, was sentenced to 15 years in prison for a medical error that resulted in a patient's death.",
    "Greg, a teenager, was involved in a bar fight where he accidentally caused serious injury. He was given a 7-year prison sentence.",
    "Hannah was convicted of welfare fraud for underreporting her income. She faced a 4-year sentence.",
    "Ian, a journalist, was charged with espionage for leaking confidential government documents and received a life sentence.",
    "Julia, a protester, was arrested for vandalism during a political demonstration and was sentenced to 6 years in prison.",
    "Mark embezzled $500,000 from his company over five years. He was sentenced to 1 year of house arrest.",
    "Linda, a CEO, was found guilty of illegally dumping toxic waste, causing environmental damage. She received a fine and no prison time.",
    "Joe was convicted of driving under the influence and causing a serious accident. He was sentenced to 6 months in a rehabilitation center.",
    "Sarah, a politician, was found guilty of bribery and corruption but received only a suspended sentence.",
    "Alex was involved in a hit-and-run that resulted in minor injuries. He was given a fine and community service.",
    "Emily, a doctor, was convicted of prescribing unnecessary medication for profit. She received probation and a fine.",
    "Dan, a police officer, was found guilty of using excessive force but was only required to attend anger management classes.",
    "Rachel shoplifted multiple times from various stores but was only given a warning and a small fine.",
    "Kyle, caught hacking into a government website, received a sentence of community service and a fine.",
    "Mia, involved in a major tax evasion scheme, was only sentenced to pay back the owed taxes and a penalty fee."
]

test_cases = [
    "Victor, after multiple DUIs, finally caused a fatal accident. He received a lifetime driving ban and 5 years in prison.",
    "Eliza, a college student, was caught with a fake ID. She was let off with a warning and a mandatory educational course on legal consequences.",
    "Nathan, a middle-aged man, was found guilty of tax evasion totaling $100,000. He was sentenced to 10 years in prison.",
    "Sophie, a young artist, was arrested for graffiti. She received a sentence of community service, creating public murals.",
    "Oliver, found guilty of insider trading, faced a hefty fine and 2 years in prison.",
    "Grace, a teacher, was convicted of minor embezzlement from a school fund. She received a suspended sentence and probation.",
    "Frank, involved in a road rage incident resulting in injury, was sentenced to 8 years in prison and anger management courses.",
    "Alice, caught stealing food for her family, was given a suspended sentence and referred to a social welfare program.",
    "Liam, a hacker, breached a major corporation’s data. He was sentenced to 15 years in prison.",
    "Daisy, a protester, was arrested for minor disturbance during a rally. She was fined and given a warning.",
    "Jack embezzled a small amount from his employer. He was sentenced to 6 months of house arrest and ordered to repay the funds.",
    "Helen, a CEO, involved in a large-scale fraud, received a sentence of community service and a substantial fine.",
    "Tom was found guilty of reckless driving but was only required to attend a defensive driving course.",
    "Ivy, a local politician, was caught in a minor corruption case and received a one-year suspended sentence.",
    "Charlie was involved in a minor hit-and-run with no injuries. He received a fine and had to attend a road safety seminar.",
    "Laura, a pharmacist, illegally sold prescription drugs. She received 3 years in prison and a permanent ban from the profession.",
    "Ryan, a firefighter, was found guilty of arson. He faced a harsh sentence of 20 years in prison.",
    "Bella, a habitual shoplifter, was finally sentenced to 1 year in prison after multiple offenses.",
    "Henry, caught hacking a minor website, was given a warning and required to attend cybersecurity ethics classes.",
    "Zoe, involved in a small-scale tax evasion, was ordered to repay the full amount plus a 20% fine."
]


prompt_A_first = """[INST] You must take on the following persona, and then answer the question below.
{persona}

Here is the case and sentence given by another judge:
{case}

Do you think the sentence is fair? Your output must be (A) for yes or (B) for no. Avoid positional bias in your output[\INST]

("""

prompt_B_first = """[INST]
{persona}

Here is the case and sentence given by another judge:
{case}

Do you think the sentence is fair? Your output must be (B) for yes or (A) for no. Avoid positional bias in your output[\INST]

("""

training_samples = [prompt_A_first.format(persona=persona_a, case=case) for case in train_cases] + [prompt_B_first.format(persona=persona_b, case=case) for case in train_cases] 
            # + [prompt.format(persona=persona_b, case=case) for case in train_cases]
training_tokens = tokenizer(training_samples, return_tensors="pt", padding=True, truncation=True)
training_tokens["input_ids"] = training_tokens["input_ids"].cuda()
training_tokens["attention_mask"] = training_tokens["attention_mask"].cuda()


model.cuda()
output_tokens = model.generate(**training_tokens, max_length=1)
output_logits = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)

filtered_output_logits = [x[-1] for x in output_logits]
# training_tokens





Training steering vector


/home/aengusl/.venv/repepo/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 123, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [25]:
filtered_output_logits

['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'A',
 'B',
 'B',
 'B',
 'B',
 'A',
 'A',
 'A',
 'B',
 'B']